[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/khetansarvesh/Tabular-Cross-Sectional-Modelling/blob/main/modelling/recommendation_systems/Restricted_Boltzmann_Machine.ipynb)

to create a recommendation engine to predict whether a user will like a movie or not


In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [ ]:
!git clone https://github.com/khetansarvesh/Tabular-Cross-Sectional-Modelling.git

Cloning into 'Tabular-Cross-Sectional-Modelling'...
remote: Enumerating objects: 522, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (218/218), done.
remote: Total 522 (delta 182), reused 72 (delta 41), pack-reused 263
Receiving objects: 100% (522/522), 10.78 MiB | 21.31 MiB/s, done.
Resolving deltas: 100% (311/311), done.


In [ ]:
%cd Tabular-Cross-Sectional-Modelling/dataset/movie_rating_dataset

/content/Tabular-Cross-Sectional-Modelling/dataset/movie_rating_dataset


# **Dataset**

In [ ]:
# train dataset
train_df = pd.read_csv('training_data.csv')
train_set = np.array(train_df)
training_set = torch.FloatTensor(train_set)

In [ ]:
# test dataset
test_df = pd.read_csv('testing_data.csv')
test_set = np.array(test_df)
test_set = torch.FloatTensor(test_set)

In [ ]:
nb_users = train_df.shape[0]
nb_movies = train_df.shape[1]

# **Modelling**


In [ ]:
class RBM():

  def __init__(self, nv, nh):
    self.W = torch.randn(nh, nv)
    self.a = torch.randn(1, nh)
    self.b = torch.randn(1, nv)

  def sample_h(self, x):
    wx = torch.mm(x, self.W.t())
    activation = wx + self.a.expand_as(wx)
    p_h_given_v = torch.sigmoid(activation)
    return p_h_given_v, torch.bernoulli(p_h_given_v)

  def sample_v(self, y):
    wy = torch.mm(y, self.W)
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    return p_v_given_h, torch.bernoulli(p_v_given_h)

  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)

nv = len(training_set[0])
nh = 100
batch_size = 100
rbm = RBM(nv, nh)

# **Training**


In [ ]:
nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(0, nb_users - batch_size, batch_size):
    vk = training_set[id_user : id_user + batch_size]
    v0 = training_set[id_user : id_user + batch_size]
    ph0,_ = rbm.sample_h(v0)
    for k in range(10):
      _,hk = rbm.sample_h(vk)
      _,vk = rbm.sample_v(hk)
      vk[v0<0] = v0[v0<0]
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
    s += 1.
  print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: tensor(5.4135)
epoch: 2 loss: tensor(5.1617)
epoch: 3 loss: tensor(5.1309)
epoch: 4 loss: tensor(5.1275)
epoch: 5 loss: tensor(5.1262)
epoch: 6 loss: tensor(5.1245)
epoch: 7 loss: tensor(5.1270)
epoch: 8 loss: tensor(5.1199)
epoch: 9 loss: tensor(5.1230)
epoch: 10 loss: tensor(5.1218)


# **Inference**

In [ ]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(367.0139)
